In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
df_fake=pd.read_csv('Fake.csv')
df_true=pd.read_csv('True.csv')

In [3]:
df_fake.shape


(23481, 4)

In [4]:
df_true.shape

(21417, 4)

In [5]:
df_fake.head(2)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"


In [6]:
df_true.head(2)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"


In [7]:
df_fake['fake']=1
df_true['fake']=0

In [8]:
df_fake.head(2)

,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1


In [9]:
df_true.head(2)

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0


In [10]:
df=pd.concat([df_fake,df_true],axis=0)

In [11]:
df

,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [12]:
df=df.drop(columns=['title','subject','date'])

In [13]:
df.head(5)

,text,fake
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [14]:
df.isnull().sum()

text    0
fake    0
dtype: int64

In [15]:
df = df.sample(frac = 1)

In [16]:
df.head(5)

,text,fake
9981,WASHINGTON (Reuters) - U.S. House Speaker Paul...,0
5548,WASHINGTON (Reuters) - President Donald Trump’...,0
10704,"1[1pro-]: relating to, caused by, or being in...",1
1765,"WASHINGTON (Reuters) - Republican Ted Cruz, a ...",0
8522,Thanks to a repeal of a long-standing law in 2...,1


In [17]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [18]:
df.head(5)

,text,fake
0,WASHINGTON (Reuters) - U.S. House Speaker Paul...,0
1,WASHINGTON (Reuters) - President Donald Trump’...,0
2,"1[1pro-]: relating to, caused by, or being in...",1
3,"WASHINGTON (Reuters) - Republican Ted Cruz, a ...",0
4,Thanks to a repeal of a long-standing law in 2...,1


In [19]:
import re
import string

In [20]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [21]:
df['text']=df['text'].apply(wordopt)

In [22]:
X=df['text']
Y=df['fake']

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [25]:
X_train.shape

(31428,)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
X_train = vectorization.fit_transform(X_train)
X_test = vectorization.transform(X_test)


In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model1=LogisticRegression()
model1.fit(X_train,y_train)

LogisticRegression()

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [30]:
y_predict=model1.predict(X_test)


In [31]:
accuracy_score(y_predict,y_test)

0.9862657757980697

In [32]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      6431
           1       0.99      0.98      0.99      7039

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [33]:
from sklearn.tree import DecisionTreeClassifier
model2=DecisionTreeClassifier()
model2.fit(X_train,y_train)

DecisionTreeClassifier()

In [34]:
y_predict2=model2.predict(X_test)
accuracy_score(y_predict2,y_test)

0.995916852264291

In [35]:
print(classification_report(y_test, y_predict2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6431
           1       1.00      1.00      1.00      7039

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [37]:
import pickle
filename='fake_news.sav'
pickle.dump(model2,open(filename,'wb'))

In [39]:
with open('vectorizer.sav', 'wb') as f:
    pickle.dump(vectorization, f)

In [40]:
df['text'].head(1).

0    washington  reuters    u s  house speaker paul...
Name: text, dtype: object